In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [ ]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist*0*')
# df_new = utils.read_df_pkl('../input/new*0*')

100%|██████████| 3/3 [00:14<00:00,  4.67s/it]


MemoryError: 

In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [ ]:
# for df in [df_hist, df_new]:
for df in [df_hist]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
#     df['hour'] = df['purchase_date'].dt.hour
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)

In [ ]:
df_hist.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)

df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7] + '-01')
df_hist['yyyy_week'] = df_hist['yyyymm'].map(lambda x: str(x)[:4]) + df_hist['weekofyear'].map(lambda x: '-0' + str(x) if len(str(x))==1 else '-' + str(x) )
df_hist.head()

### 連続week, month利用回数、期間内week, monthカバー率を特徴にする

In [89]:
df_hist.sort_values(by=[key, 'purchase_date'], inplace=True)

df_month = df_hist[[key, 'yyyymm']].drop_duplicates()
df_month['yyyymm_shift1'] = df_month.groupby(key)['yyyymm'].shift(1)
df_month['diff_month'] = ((pd.to_datetime(df_month['yyyymm']) - pd.to_datetime(df_month['yyyymm_shift1'])).dt.days)//28
# 連続してる時はフラグ1がたち、それ以外は0
df_month['continuours_flg'] = df_month['diff_month'].where(df_month['diff_month']==1, 0)

# 0になったらしきり直しなので1, 1が続く限り+1していけば、idなど関係なく、1が続いた数をカウントできる。（天才的）
cont_list = df_month['continuours_flg'].values
continuous_list = []
for i in cont_list:
    if i==0:
        cnt=1
    else:
        cnt+=1
    continuous_list.append(cnt)
print(len(cont_list))
print(len(continuous_list))

df_month['continuous'] = continuous_list
df_month['continuous_shift1'] = df_month['continuous'].shift(1)
df_month['diff_cont'] = df_month['continuous'] - df_month['continuous_shift1']
# 最大連続回数を取り出す時はこれが必要
df_month['diff_cont_shift-1'] = df_month['diff_cont'].shift(-1)

# 1月以外の連続した回数を残す
df_month.fillna(1.0, inplace=True)
df_cont = df_month[df_month['diff_cont_shift-1']<0][[key, 'yyyymm', 'continuous']]
df_cont.rename(columns={'continuous':'continuous_month'}, inplace=True)
df_cont.head()

2563765
2563765


,card_id,yyyymm,continuous_month
19095861,C_ID_00007093c1,2018-02-01,13
26539227,C_ID_0001238066,2018-02-01,6
25106417,C_ID_0001506ef0,2017-03-01,3
25106413,C_ID_0001506ef0,2018-02-01,10
12034731,C_ID_0001793786,2017-10-01,10


In [90]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

cont_max = df_cont.groupby(key)['continuous_month'].max().reset_index()
cont_min = df_cont.groupby(key)['continuous_month'].min().reset_index().rename(columns={'continuous_month':'continuous_month_min'})
cont_mean = df_cont.groupby(key)['continuous_month'].mean().reset_index().rename(columns={'continuous_month':'continuous_month_mean'})
cont_std = df_cont.groupby(key)['continuous_month'].std().reset_index().rename(columns={'continuous_month':'continuous_month_std'})

tmp_max = df_cont.merge(cont_max, how='inner', on=[key, 'continuous_month'])
cont_max_month = tmp_max.groupby(key)['yyyymm'].max()

cont_max.rename(columns={'continuous_month':'continuous_month_max'}, inplace=True)

cont_max_month = cont_max_month.to_frame().reset_index().rename(columns={'index':key})
cont_max_month = cont_max_month.merge(train_test[[key, 'first_active_month']], how='left', on=key)
cont_max_month = cont_max_month.merge(cont_max, how='left', on=key)

cont_max_month['diff_cont_max_month_from_today'] = ((datetime.datetime.today() - pd.to_datetime(cont_max_month['yyyymm'])).dt.days)//27
cont_max_month['diff_cont_max_last_month_from_first'] = ((pd.to_datetime(cont_max_month['yyyymm']) - pd.to_datetime(cont_max_month['first_active_month']) ).dt.days)//30
cont_max_month['diff_cont_max_from_first'] = cont_max_month['diff_cont_max_last_month_from_first'] - cont_max_month['continuous_month_max']
cont_max_month['diff_cont_max_from_first'] += 1

cont_cnt = df_cont.groupby(key).size()
cont_cnt.name = 'cont_month_cut_cnt'
cont_cnt = cont_cnt.to_frame().reset_index().rename(columns={'index':key})


# contがついてるfeatureのみ保存すればOK
# for feat in [cont_min, cont_mean, cont_std, cont_max_month]:
for feat in [cont_cnt]:
    df_train = df_train.merge(feat, how='left', on=key)
    df_test = df_test.merge(feat, how='left', on=key)

100%|██████████| 3/3 [00:00<00:00, 173.92it/s]


### 連続月達成時期を1月ごとに横持ちする（01）で
2ヵ月連続を達成したのは何か月前か？という形で  

In [65]:
tmp_list = []
for i in range(2, 25, 1):
    tmp = df_cont[df_cont['continuous_month']>=i].groupby(key)['yyyymm'].max().reset_index()
    tmp[f'diff_month_cont_month'] = (datetime.datetime.today() - pd.to_datetime(tmp['yyyymm'])).dt.days//30
    tmp[f'cont_month_timing_from_today'] = f'cont_{i}month_timing_from_today'
    tmp_list.append(tmp[[key, f'cont_month_timing_from_today', f'diff_month_cont_month']].copy())
cont_from = pd.concat(tmp_list, axis=0)
display(cont_from.head(10))
display(cont_from.tail(10))

,card_id,cont_month_timing_from_today,diff_month_cont_month
0,C_ID_00007093c1,cont_2month_timing_from_today,11
1,C_ID_0001238066,cont_2month_timing_from_today,11
2,C_ID_0001506ef0,cont_2month_timing_from_today,11
3,C_ID_0001793786,cont_2month_timing_from_today,15
4,C_ID_000183fdda,cont_2month_timing_from_today,11
5,C_ID_00024e244b,cont_2month_timing_from_today,11
6,C_ID_0002709b5a,cont_2month_timing_from_today,11
7,C_ID_00027503e2,cont_2month_timing_from_today,12
8,C_ID_000298032a,cont_2month_timing_from_today,12
9,C_ID_0002ba3c2e,cont_2month_timing_from_today,15


,card_id,cont_month_timing_from_today,diff_month_cont_month
35275,C_ID_ffefdb10d6,cont_14month_timing_from_today,11
35276,C_ID_fff04976bb,cont_14month_timing_from_today,11
35277,C_ID_fff184faf3,cont_14month_timing_from_today,11
35278,C_ID_fff690bccc,cont_14month_timing_from_today,11
35279,C_ID_fff6b4456b,cont_14month_timing_from_today,11
35280,C_ID_fffadd3551,cont_14month_timing_from_today,11
35281,C_ID_fffc96bf24,cont_14month_timing_from_today,11
35282,C_ID_fffd1207a8,cont_14month_timing_from_today,11
35283,C_ID_fffe78b232,cont_14month_timing_from_today,11
35284,C_ID_ffff828181,cont_14month_timing_from_today,11


In [67]:
cont_from_table = cont_from.pivot_table(index=key, columns='cont_month_timing_from_today', values='diff_month_cont_month')
cont_from_table.head()

cont_month_timing_from_today,cont_10month_timing_from_today,cont_11month_timing_from_today,cont_12month_timing_from_today,cont_13month_timing_from_today,cont_14month_timing_from_today,cont_2month_timing_from_today,cont_3month_timing_from_today,cont_4month_timing_from_today,cont_5month_timing_from_today,cont_6month_timing_from_today,cont_7month_timing_from_today,cont_8month_timing_from_today,cont_9month_timing_from_today
card_id,,,,,,,,,,,,,
C_ID_00007093c1,11.0,11.0,11.0,11.0,NaN,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,11.0,11.0,11.0,11.0,11.0,NaN,NaN,NaN
C_ID_0001506ef0,11.0,NaN,NaN,NaN,NaN,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
C_ID_0001793786,15.0,NaN,NaN,NaN,NaN,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,11.0,11.0,11.0,11.0,11.0,11.0,NaN,NaN


In [68]:
df_train = df_train.merge(cont_from_table, how='left', on=key)
df_test = df_test.merge(cont_from_table, how='left', on=key)
print(df_train.shape)
df_train.head()

(201917, 28)


,first_active_month_x,card_id,feature_1,feature_2,feature_3,target,continuous_month_min,continuous_month_mean,continuous_month_std,yyyymm,first_active_month_y,continuous_month_max,diff_cont_max_month_from_today,diff_cont_max_last_month_from_first,diff_cont_max_from_first,cont_10month_timing_from_today,cont_11month_timing_from_today,cont_12month_timing_from_today,cont_13month_timing_from_today,cont_14month_timing_from_today,cont_2month_timing_from_today,cont_3month_timing_from_today,cont_4month_timing_from_today,cont_5month_timing_from_today,cont_6month_timing_from_today,cont_7month_timing_from_today,cont_8month_timing_from_today,cont_9month_timing_from_today
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,9.0,9.0,NaN,2018-02-01,2017-06,9.0,12.0,8.0,0.0,NaN,NaN,NaN,NaN,NaN,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,13.0,13.0,NaN,2018-01-01,2017-01,13.0,13.0,12.0,0.0,12.0,12.0,12.0,12.0,NaN,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,3.0,4.0,1.0,2017-05-01,2016-08,5.0,22.0,9.0,5.0,NaN,NaN,NaN,NaN,NaN,11.0,11.0,15.0,20.0,NaN,NaN,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,6.0,6.0,NaN,2018-02-01,2017-09,6.0,12.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,11.0,11.0,11.0,11.0,11.0,NaN,NaN,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,4.0,4.0,NaN,2018-02-01,2017-11,4.0,12.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,11.0,11.0,11.0,NaN,NaN,NaN,NaN,NaN


In [91]:
fname = '134_com'
ignore_features = ['first_active_month', 'card_id']

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('cont')):continue
#     if not(col.count('trans')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

### 135 week

In [83]:
# df_hist.sort_values(by=[key, 'purchase_date'], inplace=True)
# df_week = df_hist[[key, 'yyyy_week']].drop_duplicates()

# week_val = df_week['yyyy_week'].drop_duplicates().sort_values().to_frame()
# week_val['index'] = np.arange(len(week_val)) + 1
# df_week = df_week.merge(week_val, how='inner', on='yyyy_week')

df_week.sort_values(by=[key, 'yyyy_week'], inplace=True)
df_week['index_shift1'] = df_week.groupby(key)['index'].shift(1)
df_week['diff_week'] = df_week['index'] - df_week['index_shift1']
# 連続してる時はフラグ1がたち、それ以外は0
df_week['continuours_flg'] = df_week['diff_week'].where(df_week['diff_week']==1, 0)

# 0になったらしきり直しなので1, 1が続く限り+1していけば、idなど関係なく、1が続いた数をカウントできる。（天才的）
cont_list = df_week['continuours_flg'].values
continuous_list = []
for i in cont_list:
    if i==0:
        cnt=1
    else:
        cnt+=1
    continuous_list.append(cnt)
print(len(cont_list))
print(len(continuous_list))

df_week['continuous'] = continuous_list
df_week['continuous_shift1'] = df_week['continuous'].shift(1)
df_week['diff_cont'] = df_week['continuous'] - df_week['continuous_shift1']
# 最大連続回数を取り出す時はこれが必要
df_week['diff_cont_shift-1'] = df_week['diff_cont'].shift(-1)

# 1月以外の連続した回数を残す
df_week.fillna(1.0, inplace=True)
df_cont = df_week[df_week['diff_cont_shift-1']<0][[key, 'yyyy_week', 'continuous']]
df_cont.rename(columns={'continuous':'continuous_week'}, inplace=True)
df_cont.head()

,card_id,yyyy_week,continuous_week
309654,C_ID_00007093c1,2017-11,5
1458189,C_ID_00007093c1,2017-29,11
2168729,C_ID_00007093c1,2017-36,6
3127120,C_ID_00007093c1,2017-45,7
3756188,C_ID_00007093c1,2017-50,4


In [86]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

cont_max = df_cont.groupby(key)['continuous_week'].max().reset_index().rename(columns={'continuous_week':'continuous_week_max'})
cont_min = df_cont.groupby(key)['continuous_week'].min().reset_index().rename(columns={'continuous_week':'continuous_week_min'})
cont_mean = df_cont.groupby(key)['continuous_week'].mean().reset_index().rename(columns={'continuous_week':'continuous_week_mean'})
cont_std = df_cont.groupby(key)['continuous_week'].std().reset_index().rename(columns={'continuous_week':'continuous_week_std'})

cont_cnt = df_cont.groupby(key).size()
cont_cnt.name = 'cont_week_cut_cnt'
cont_cnt = cont_cnt.to_frame().reset_index().rename(columns={'index':key})

# contがついてるfeatureのみ保存すればOK
for feat in [cont_max, cont_min, cont_mean, cont_std, cont_cnt]:
    df_train = df_train.merge(feat, how='left', on=key)
    df_test = df_test.merge(feat, how='left', on=key)
    
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 175.15it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3,target,continuous_week_max,continuous_week_min,continuous_week_mean,continuous_week_std,cont_week_cut_cnt
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,35.0,35.0,35.000000,NaN,1.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,33.0,9.0,18.333333,12.858201,3.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,7.0,2.0,3.750000,2.362908,4.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,11.0,3.0,6.333333,4.163332,3.0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,17.0,17.0,17.000000,NaN,1.0


In [88]:
fname = '135_cow'
ignore_features = ['first_active_month', 'card_id']

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('cont')):continue
#     if not(col.count('trans')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)